In [52]:
#Import all ecessary libraries
import requests 
from bs4 import BeautifulSoup
from flask import request
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [64]:
#To extract reviews from multiple pages
all_reviews = []
for page_num in range(1, 20):

    url_source = 'https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100&page={page_num}'
    url=url_source.format(page_num=page_num)
    r=requests.get(url)

#extract data using BeautifulSoup library
soup=BeautifulSoup(r.content, 'lxml')

reviews = soup.find_all('h2', {'class': "text_header"})
div_reviews = soup.find_all('div', {'class': "text_content"})
                                    
#To add the reviews of the next page one after the other
review2 = ' '
for review in reviews:
    review2 += review.text.strip() + '\n'
                                    
for div_review in div_reviews:
    review2 += div_review.text.strip() + '\n'
all_reviews.append(review2)
                                    
print(all_reviews)

[' "not improving where they should"\n"getting a refund is a nightmare"\n"How far BA has sunk!"\n"I was very disappointed"\n"BA has lost two loyal customers"\n"Good value if a free upgrade"\n"not the BA that people once knew"\n"staff could not have cared less"\n"service is consistently unacceptable"\n"the flight was enjoyable"\n"a very cheap offer to upgrade"\n"my first choice out of City"\n"Staff are polite and friendly"\n"a surly disapproving stewardess"\n"game over for me"\n"cost cutting affects all cabins"\n"downgraded their economy service"\n"So low cost. So over them"\n"An absolute disgrace"\n"Rude and aggressive customer service"\n"is simply a budget airline"\n"a poor service all round "\n"I have lost my patience"\n"Absolutely useless"\n"never seen an airline deteriorate so quickly"\n"things are not as they were"\n"Service was an absolute shambles"\n"pleasant and enjoyable experience"\n"should not call this business class"\n"friendly, attentive and helpful"\n"what you expect fro

In [60]:
#Extract into a csv file
for review in all_reviews:
    with open('review.csv', 'a', encoding = 'utf-8') as f:
        f.write(review + '\n')

In [44]:
#If needed, you may perform the following installations: 
#pip install --upgrade vaderSentiment
#pip install --upgrade textblob
#pip install --upgrade tokenize

In [61]:
#For Sentiment Analysis
import re
import pandas as pd
import nltk

#File path:
#"C:\Users\USER\Documents\Jupyter Workings\review.csv"

from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from tqdm.notebook import tqdm
import seaborn as sns
from nltk.tokenize import word_tokenize

#Download Stopwrds
nltk.downloader.download('stopwords')
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('punkt')

#Load input dataframe
df = pd.read_csv(r"C:\\Users\\USER\\Documents\\Jupyter Workings\\review.csv")

#Load stopwords
stop_words = set(stopwords.words('english'))

#Initialize sentiment analyzer
vader = SentimentIntensityAnalyzer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 8101: invalid start byte

In [65]:
df.to_csv(r"C:\Users\USER\Documents\Jupyter Workings\review.csv", index = [False])

In [47]:
#Load the new input dataframe
#df = pd.read_csv(r"C:\\Users\\USER\\Documents\\Jupyter Workings\\review.csv")

In [66]:
df.head()

,Unnamed: 0,"""no sandwiches left to purchase"""
0,0,another very good flight
1,1,don't provide free food and drink
2,2,way below expected service
3,3,onboard charity collection is inappropriate
4,4,staff were so friendly


In [49]:
#Define regex pattern to match unwanted characters
pattern = r'[^A-Za-z\s]+'

#create empty output dataframe
output_df = pd.DataFrame(columns=["ID", "REVIEWS", "POSITIVE_SCORE", "NEGATIVE_SCORE", 
                                  "SENTIMENT", "SUBJECTIVITY_SCORE"])
output_df.head()

,ID,REVIEWS,POSITIVE_SCORE,NEGATIVE_SCORE,SENTIMENT,SUBJECTIVITY_SCORE


In [50]:
import pandas as pd
from tqdm import tqdm

#Loop through rows of input dataframe
for index, row in df.iterrows():
    ID = row["ID"]
    REVIEWS = row["REVIEWS"]
    
#Apply regex to remove unwanted characters
cleaned_text = re.sub(pattern, " ", str(REVIEWS))

#Tokenize text into words
words = nltk.word_tokenize(cleaned_text)

#Remove stopwords and lowercase
words = [word.lower() for words in words if word.lower() not in stop_words]

#Joined words back into cleaned text
cleaned_text = ' ' .join(words)

#Get polarity scores for cleaned text 
scores = vader.polarity_scores(cleaned_text)

#Add compound score to scores dictionary
scores["compound"] = scores["pos"] - scores["neg"]

#To get subjective scores
blob = TextBlob(cleaned_text)
polarity_score = blob.sentiment.polarity
subjectivity_score = blob.sentiment.subjectivity

#Append results to outpout dataframe
#output_df = pd.concat([output_df, pd.DataFrame({"ID": ID, "REVIEWS": REVIEWS, "POSITIVE_SCORE": 
                                               # scores["pos"], "NEGATICE_SCORE": scores["neg"], 
                                                #"SENTIMENT": scores["compound"], "SUBJECTIVITY_SCORE": 
                                               # subjectivity_score}, index=[0])])
output_df = pd.DataFrame({"ID": ID, "REVIEWS": REVIEWS, "POSITIVE_SCORE": scores["pos"], "NEGATICE_SCORE": 
                          scores["neg"], "SENTIMENT": scores["compound"], "SUBJECTIVITY_SCORE": 
                          subjectivity_score}, index = [0])

KeyError: 'ID'

In [ ]:
out = pd.concat([df, output_df])

In [ ]:
out.head()

In [ ]:
pd.DataFrame(out)